```{autolink-concat}
```

::::{margin}
:::{card} Amplitude building with K-matrix dynamics
TR-030
:::
::::

# Fit amplitude model with P-vector dynamics

In [ ]:
%pip install -q 'qrules[viz]==0.10.2' 'tensorwaves[jax,phsp]==0.4.12' ampform==0.15.4 pandas==2.2.2 sympy==1.12

In [ ]:
from __future__ import annotations

import os
import re
from collections import defaultdict
from typing import Any

import ampform
import attrs
import graphviz
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import qrules
import sympy as sp
from ampform.dynamics.builder import TwoBodyKinematicVariableSet
from ampform.helicity import ParameterValues
from ampform.io import aslatex
from ampform.kinematics.phasespace import Kallen
from ampform.sympy import unevaluated
from IPython.display import Latex, display
from matplotlib import cm
from qrules.particle import Particle, ParticleCollection
from sympy import Abs
from tensorwaves.data import (
    IntensityDistributionGenerator,
    SympyDataTransformer,
    TFPhaseSpaceGenerator,
    TFUniformRealNumberGenerator,
    TFWeightedPhaseSpaceGenerator,
)
from tensorwaves.estimator import UnbinnedNLL
from tensorwaves.function.sympy import create_parametrized_function
from tensorwaves.interface import DataSample, ParametrizedFunction
from tensorwaves.optimizer import Minuit2
from tensorwaves.optimizer.callbacks import CSVSummary

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Collect dynamics symbols

| Resonance | $m$ [MeV] | $\Gamma$ [MeV] | $J^P$ |
|-----------|-----------|----------------|-------|
| $N^*(1440)$ | 1398      | 167            | $\frac{1}{2}^{+}$ |
| $N^*(1535)$ | 1530      | 210            | $\frac{1}{2}^{-}$ |
| $N^*(1650)$ | 1668      | 194            | $\frac{1}{2}^{-}$ |
| $N^*(1710)$ | 1749      | 263            | $\frac{1}{2}^{+}$ |
| $N^*(1880)$ | 1876      | 261            | $\frac{1}{2}^{+}$ |
| $N^*(1895)$ | 2045      | 240            | $\frac{1}{2}^{-}$ |
| $N^*(1900)$ | 1970      | 255            | $\frac{3}{2}^{+}$ |

In [ ]:
def create_dynamics_symbol(
    resonance: Particle, variable_pool: TwoBodyKinematicVariableSet
) -> tuple[sp.Expr, dict[sp.Symbol, float]]:
    J = sp.Rational(resonance.spin)
    Q = resonance.charge
    P = sp.Rational(resonance.parity)
    if variable_pool.angular_momentum is not None:
        L = sp.Rational(variable_pool.angular_momentum)
        X = sp.Symbol(Rf"X_{{Q={Q:+d}, S={J}, P ={P}}}^{{l={L}}}")
    else:
        X = sp.Symbol(Rf"X_{{Q={Q:+d}, S={J}, P ={P}}}")
    COLLECTED_X_SYMBOLS[X].add((resonance, variable_pool))
    parameter_defaults = {}
    return X, parameter_defaults


COLLECTED_X_SYMBOLS = defaultdict(set)

In [ ]:
def load_particle_database() -> ParticleCollection:
    particle_database = qrules.load_default_particles()
    additional_definitions = qrules.io.load("030/additional-definitions.yml")
    particle_database.update(additional_definitions)
    return particle_database


PARTICLE_DB = load_particle_database()

In [ ]:
reaction = qrules.generate_transitions(
    initial_state="J/psi(1S)",
    final_state=["eta", "p", "p~"],
    allowed_intermediate_particles=[
        "N(Fakestar2)+",
        "N(1650)+",
        "N(1900)+",
        "N(Fakestar)+",
    ],
    allowed_interaction_types=["strong"],
    formalism="helicity",
    particle_db=PARTICLE_DB,
)
dot = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(dot)

In [ ]:
model_builder = ampform.get_builder(reaction)
model_builder.adapter.permutate_registered_topologies()
model_builder.scalar_initial_state_mass = True
model_builder.stable_final_state_ids = [0, 1, 2]
for name in reaction.get_intermediate_particles().names:
    model_builder.set_dynamics(name, create_dynamics_symbol)
model = model_builder.formulate()

In [ ]:
selected_amplitudes = {
    k: v for i, (k, v) in enumerate(model.amplitudes.items()) if i < 3
}
src = aslatex(selected_amplitudes)

## Formulate dynamics expression

In [ ]:
for symbol, resonances in COLLECTED_X_SYMBOLS.items():
    display(symbol)
    for p, _ in resonances:
        print(f"  {p.name:<20s} {p.mass:>8g} GeV  {p.width:>8g} GeV ")
model.parameter_defaults

## Formulate Dynamics

### Phasespace factor

In [ ]:
@unevaluated(real=False)
class PhaseSpaceCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}_{{{m1},{m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log(Abs((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2)))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))


@unevaluated(real=False)
class ChannelWidth(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    gamma_R: Any
    _latex_repr_ = R"\Gamma_s\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2, gamma_R = self.args
        return gamma_R * PhaseSpaceCM(s, m1, m2)

### Relativistic Breit-Wigner

In [ ]:
PARAMETERS_BW = {}
PARAMETERS_BW.update(model.parameter_defaults)


def formulate_rel_bw(
    resonances: list[tuple[Particle, TwoBodyKinematicVariableSet]],
) -> sp.Expr:
    (_, variables), *_ = resonances
    s = variables.incoming_state_mass**2
    m_a = variables.outgoing_state_mass1
    m_b = variables.outgoing_state_mass2
    w = [sp.Symbol(Rf"w_{{{p.latex}}}") for p, _ in resonances]
    m = [sp.Symbol(Rf"m_{{{p.latex}}}") for p, _ in resonances]
    b = [sp.Symbol(Rf"b_{{{p.latex}}}") for p, _ in resonances]
    d = [sp.Symbol(Rf"d_{{{p.latex}}}") for p, _ in resonances]
    L = [sp.Symbol(Rf"L_{{{p.latex}}}") for p, _ in resonances]
    w_s = (ChannelWidth(s, m_a, m_b, w_) for w_ in w)
    rel_bw = sum((w_ * m_) / (m_**2 - s - m_ * w_s_) for m_, w_, w_s_ in zip(m, w, w_s))
    for i, (resonance, _) in enumerate(resonances):
        PARAMETERS_BW[w[i]] = resonance.width
        PARAMETERS_BW[m[i]] = resonance.mass
        PARAMETERS_BW[b[i]] = 1
        PARAMETERS_BW[d[i]] = 1
        PARAMETERS_BW[L[i]] = 0
    return rel_bw

### $K$ matrix 

In [ ]:
PARAMETERS_F = {}
PARAMETERS_F.update(model.parameter_defaults)


def formulate_K_matrix(
    resonances: list[tuple[Particle, TwoBodyKinematicVariableSet]],
) -> sp.Expr:
    (_, variables), *_ = resonances
    s = variables.incoming_state_mass**2
    g = [sp.Symbol(Rf"g_{{{p.latex}}}") for p, _ in resonances]
    m = [sp.Symbol(Rf"m_{{{p.latex}}}") for p, _ in resonances]

    kmatrix = sum((g_**2) / (m_**2 - s) for m_, g_ in zip(m, g))
    for i, (resonance, _) in enumerate(resonances):
        PARAMETERS_F[m[i]] = resonance.mass
        PARAMETERS_F[g[i]] = 1
    return kmatrix

### $P$ vector

In [ ]:
def formulate_P_vector(
    resonances: list[tuple[Particle, TwoBodyKinematicVariableSet]],
) -> sp.Expr:
    (_, variables), *_ = resonances
    s = variables.incoming_state_mass**2
    g = [sp.Symbol(Rf"g_{{{p.latex}}}") for p, _ in resonances]
    m = [sp.Symbol(Rf"m_{{{p.latex}}}") for p, _ in resonances]
    beta = [sp.Symbol(Rf"\beta_{{{p.latex}}}") for p, _ in resonances]
    P_vector = sum((g_ * beta_) / (m_**2 - s) for m_, g_, beta_ in zip(m, g, beta))
    for i, (resonance, _) in enumerate(resonances):
        PARAMETERS_F[m[i]] = resonance.mass
        PARAMETERS_F[beta[i]] = 1 + 0j
        PARAMETERS_F[g[i]] = 1
    return P_vector

### $F$ vector

In [ ]:
def formulate_F_vector(
    resonances: list[tuple[Particle, TwoBodyKinematicVariableSet]],
) -> sp.Expr:
    (_, variables), *_ = resonances
    s = variables.incoming_state_mass**2
    m_a = variables.outgoing_state_mass1
    m_b = variables.outgoing_state_mass2
    rho = PhaseSpaceCM(s, m_a, m_b)
    K = formulate_K_matrix(resonances)
    P = formulate_P_vector(resonances)
    return (1 / (1 - rho * K)) * P

### Model relativistic Breit-Wigner

In [ ]:
dynamics_expressions_rel_bw = {
    symbol: formulate_rel_bw(resonances)
    for symbol, resonances in COLLECTED_X_SYMBOLS.items()
}
model_rel_bw = attrs.evolve(
    model,
    parameter_defaults=ParameterValues({
        **model.parameter_defaults,
        **PARAMETERS_BW,
    }),
)

In [ ]:
full_expression_rel_bw = model_rel_bw.expression.doit().xreplace(
    dynamics_expressions_rel_bw
)

### Model $F$ vector

In [ ]:
dynamics_expressions_fvector = {
    symbol: formulate_F_vector(resonances)
    for symbol, resonances in COLLECTED_X_SYMBOLS.items()
}
model_fvector = attrs.evolve(
    model,
    parameter_defaults=ParameterValues({
        **model.parameter_defaults,
        **PARAMETERS_F,
    }),
)
Latex(aslatex(dynamics_expressions_fvector))

In [ ]:
model_fvector.parameter_defaults

In [ ]:
full_expression_fvector = model_fvector.expression.doit().xreplace(
    dynamics_expressions_fvector
)
sp.count_ops(full_expression_fvector)

### Create Parametrized Function


In [ ]:
unfolded_expression_rel_bw = full_expression_rel_bw.doit()

intensity_func_rel_bw = create_parametrized_function(
    expression=unfolded_expression_rel_bw,
    backend="jax",
    parameters=PARAMETERS_BW,
)

In [ ]:
unfolded_expression_fvector = full_expression_fvector.doit()

intensity_func_fvector = create_parametrized_function(
    expression=unfolded_expression_fvector,
    backend="jax",
    parameters=PARAMETERS_F,
)

## Update parameters

In [ ]:
new_parameters_fvector = {
    R"m_{N(Fakestar)^+}": 1.95,
    R"\beta_{N(Fakestar)^+}": 1 + 0j,
    R"m_{N(1900)^+}": 1.9,
    R"\beta_{N(1900)^+}": 1 + 0j,
    R"g_{N(1900)^+}": 1,
    R"g_{N(Fakestar)^+}": 1,
    R"m_{N(Fakestar2)^+}": 1.75,
    R"\beta_{N(Fakestar2)^+}": 1 + 0j,
    R"m_{N(1650)^{+}}": 1.65,
    R"\beta_{N(1650)^{+}}": 1 + 0j,
    R"g_{N(1650)^{+}}": 1.65,
    R"g_{N(Fakestar2)^+}": 1,
}

new_parameters_bw = {
    R"m_{N(Fakestar)^+}": 1.85,
    R"w_{N(Fakestar)^+}": 1 / 1.85,
    R"m_{N(1900)^+}": 1.9,
    R"w_{N(1900)^+}": 1 / 1.9,
    R"m_{N(Fakestar2)^+}": 1.75,
    R"w_{N(Fakestar2)^+}": 1 / 1.75,
    R"m_{N(1650)^{+}}": 1.65,
    R"w_{N(1650)^{+}}": 1 / 1.65,
}

In [ ]:
intensity_func_fvector.update_parameters(new_parameters_fvector)
intensity_func_rel_bw.update_parameters(new_parameters_bw)
intensity_func_fvector.parameters

##  Generate data with $F$ vector
### Generate phase space sample

In [ ]:
helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables, backend="numpy"
)

In [ ]:
rng = TFUniformRealNumberGenerator(seed=0)
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=reaction.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction.final_state.items()},
)
phsp_momenta = phsp_generator.generate(100_000, rng)
phsp = helicity_transformer(phsp_momenta)
phsp = {k: v.real for k, v in phsp.items()}
phsp

## Plot Sub-Intensities

In [ ]:
def compute_sub_intensity(
    func: ParametrizedFunction,
    input_data: DataSample,
    resonances: list[str],
    coupling_pattern: str = r"(\\beta|g)",
):
    original_parameters = dict(func.parameters)
    negative_lookahead = f"(?!{'|'.join(map(re.escape, resonances))})"
    # https://regex101.com/r/WrgGyD/1
    pattern = rf"^{coupling_pattern}({negative_lookahead}.)*$"
    set_parameters_to_zero(func, pattern)
    array = func(input_data)
    func.update_parameters(original_parameters)
    return array


def set_parameters_to_zero(func: ParametrizedFunction, name_pattern: str) -> None:
    new_parameters = dict(func.parameters)
    for par_name in func.parameters:
        if re.match(name_pattern, par_name) is not None:
            new_parameters[par_name] = 0
    func.update_parameters(new_parameters)

In [ ]:
total_intensities = intensity_func_fvector(phsp)
total_intensities_1 = intensity_func_rel_bw(phsp)
sub_intensities = {
    p: compute_sub_intensity(
        intensity_func_fvector, phsp, resonances=[p.latex], coupling_pattern=r"\\beta"
    )
    for symbol, resonances in COLLECTED_X_SYMBOLS.items()
    for p, _ in resonances
}
sub_intensities_bw = {
    p: compute_sub_intensity(intensity_func_fvector, phsp, resonances=[p.latex])
    for symbol, resonances in COLLECTED_X_SYMBOLS.items()
    for p, _ in resonances
}

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5), dpi=300)
ax.set_xlim(2, 5)
ax.set_xlabel(R"$m_{p\eta}^{2}$ [GeV^{2}]")
ax.set_xlabel(R"Intensity [a. u.]")
ax.set_yticks([])

bins = 150
phsp_projection = np.real(phsp["m_01"]) ** 2
ax.hist(
    phsp_projection,
    weights=total_intensities,
    bins=bins,
    alpha=0.2,
    color="hotpink",
    label="Full intensity",
)
ax.hist(
    phsp_projection,
    weights=total_intensities_1,
    bins=bins,
    alpha=0.2,
    color="grey",
    label="Full intensity",
)
ax.hist(
    len(sub_intensities) * [phsp_projection],
    weights=list(sub_intensities.values()),
    bins=bins,
    alpha=0.6,
    label=[
        Rf"Resonance at ${p.mass}\,\mathrm{{GeV^2}}$ $F$ vector"
        for p in sub_intensities
    ],
    histtype="step",
)

ax.hist(
    len(sub_intensities_bw) * [phsp_projection],
    weights=list(sub_intensities_bw.values()),
    bins=bins,
    alpha=0.6,
    label=[
        Rf"Resonance at ${p.mass}\,\mathrm{{GeV^2}}$ Breit-Wigner"
        for p in sub_intensities
    ],
    histtype="step",
    ls="dotted",
)

fig.legend(loc="upper right")
plt.tight_layout()
plt.show()

### Dynamics expressions

In [ ]:
dynamics_expr_rel_bw, *_ = dynamics_expressions_rel_bw.values()
dynamics_expr_rel_bw

In [ ]:
dynamics_expr_fvector, *_ = dynamics_expressions_fvector.values()
dynamics_expr_fvector

In [ ]:
dynamics_func_bw = create_parametrized_function(
    expression=dynamics_expr_rel_bw.doit(),
    backend="numpy",
    parameters=model_rel_bw.parameter_defaults,
    use_cse=False,
)

In [ ]:
dynamics_func_fvector = create_parametrized_function(
    expression=dynamics_expr_fvector.doit(),
    backend="numpy",
    parameters=model_fvector.parameter_defaults,
    use_cse=False,
)

### Weighted data with $F$ vector 

In [ ]:
figD, axD = plt.subplots(figsize=(6, 5))
c = axD.hist(
    np.real(phsp["m_01"]) ** 2,
    bins=100,
    weights=np.real(intensity_func_rel_bw(phsp)),
)

axD.set_xlabel(R"$M^2\left(\eta p\right)\, \mathrm{[(GeV/c)^2]}$")
axD.set_ylabel(R"Intensity [a.u.]")
figD.tight_layout()
plt.show()
phsp["m_01"]

In [ ]:
weighted_phsp_generator = TFWeightedPhaseSpaceGenerator(
    initial_state_mass=model.reaction_info.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in model.reaction_info.final_state.items()},
)
data_generator = IntensityDistributionGenerator(
    domain_generator=weighted_phsp_generator,
    function=intensity_func_rel_bw,
    domain_transformer=helicity_transformer,
)
data_momenta = data_generator.generate(50_000, rng)
pd.DataFrame({
    (k, label): np.transpose(v)[i]
    for k, v in data_momenta.items()
    for i, label in enumerate(["E", "px", "py", "pz"])
})
phsp = helicity_transformer(phsp_momenta)
data = helicity_transformer(data_momenta)
data_frame = pd.DataFrame(data)
phsp_frame = pd.DataFrame(phsp)

In [ ]:
resonances = sorted(
    model.reaction_info.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(
    0, 1, len(intensity_func_fvector.parameters.items())
)
colors = [cm.rainbow(x) for x in evenly_spaced_interval]
fig, ax = plt.subplots(figsize=(9, 4))
ax.hist(
    np.real(data_frame["m_01"]),
    bins=200,
    alpha=0.5,
    density=True,
)
ax.set_xlabel("$m$ [GeV]")
for (k, v), color in zip(new_parameters_bw.items(), colors):
    if k.startswith("m_{"):
        ax.axvline(
            x=v,
            linestyle="dotted",
            label=r"$" + k + "$",
            color=color,
        )
ax.legend()
plt.show()
# Multiply

### Perform fit

#### Define estimator

In [ ]:
def safe_downcast_to_real(data: DataSample) -> DataSample:
    return {
        key: array.real if np.isrealobj(array) else array for key, array in data.items()
    }


data_real = safe_downcast_to_real(data)
phsp_real = safe_downcast_to_real(phsp)

In [ ]:
estimator_bw = UnbinnedNLL(
    intensity_func_rel_bw,
    data=data_real,
    phsp=phsp_real,
    backend="jax",
)

estimator_fvector = UnbinnedNLL(
    intensity_func_fvector,
    data=data_real,
    phsp=phsp_real,
    backend="jax",
)

In [ ]:
reaction_info = model.reaction_info
resonances = sorted(
    reaction_info.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval_F = np.linspace(
    0, 1, len(intensity_func_fvector.parameters.items())
)
colors_F = [cm.rainbow(x) for x in evenly_spaced_interval_F]
evenly_spaced_interval_BW = np.linspace(
    0, 1, len(intensity_func_rel_bw.parameters.items())
)
colors_BW = [cm.gist_rainbow(x) for x in evenly_spaced_interval_BW]


def indicate_masses(ax):
    ax.set_xlabel("$m$ [GeV]")
    for (k, v), color_F in zip(intensity_func_fvector.parameters.items(), colors_F):
        if k.startswith("m_{"):
            ax.axvline(
                x=v,
                linestyle="dotted",
                label=r"$" + k + "$" "(F vector)",
                color=color_F,
            )
    for (k, v), color_BW in zip(intensity_func_rel_bw.parameters.items(), colors_BW):
        if k.startswith("m_{"):
            ax.axvline(
                x=v,
                linestyle="dotted",
                label=r"$" + k + "$" "(Breit-Wigner)",
                color=color_BW,
            )


def compare_model(
    variable_name,
    data,
    phsp,
    function1,
    function2,
    bins=100,
):
    intensities1 = function1(phsp)
    intensities2 = function2(phsp)
    _, ax = plt.subplots(figsize=(9, 4))
    data_projection = np.real(data[variable_name])
    ax = plt.gca()
    ax.hist(
        data_projection,
        bins=bins,
        alpha=0.5,
        label="data",
        density=True,
    )
    phsp_projection = np.real(phsp[variable_name])
    ax.hist(
        phsp_projection,
        weights=np.array(intensities1),
        bins=bins,
        histtype="step",
        color="red",
        label="Fit model with K matrix",
        density=True,
    )
    ax.hist(
        phsp_projection,
        weights=np.array(intensities2),
        bins=bins,
        histtype="step",
        color="blue",
        label="Fit model with Breit Wigner",
        density=True,
    )
    indicate_masses(ax)
    ax.legend()

### Set initial parameters

In [ ]:
m_1900 = 1.93
beta_1900 = 0.9 + 0j
g_1900 = 1.0
m_1650 = 1.65
beta_1650 = 1 + 0j
g_1900 = 1.0
m_Fakestar2 = 1.5
beta_Fakestar2 = 1 + 0j
g_Fakestar2 = 1.0
m_Fakestar1 = 1.94
initial_parameters_fvector = {
    R"m_{N(Fakestar)^+}": 1.95,
    R"\beta_{N(Fakestar)^+}": 0.9 + 0j,
    R"m_{N(1900)^+}": 1.91,
    R"\beta_{N(1900)^+}": 1 + 0j,
    R"g_{N(1900)^+}": 1.0,
    R"g_{N(Fakestar)^+}": 1.0,
    R"m_{N(Fakestar2)^+}": 1.7,
    R"\beta_{N(Fakestar2)^+}": 1 + 0j,
    R"m_{N(1650)^{+}}": 1.67,
    R"\beta_{N(1650)^{+}}": 1 + 0j,
    R"g_{N(1650)^{+}}": 1.6,
    R"g_{N(Fakestar2)^+}": 1,
}

initial_parameters_bw = {
    R"m_{N(Fakestar)^+}": 1.8,
    R"w_{N(Fakestar)^+}": 1 / 1.85,
    R"m_{N(1900)^+}": 1.93,
    R"w_{N(1900)^+}": 1 / 1.93,
    R"m_{N(Fakestar2)^+}": 1.7,
    R"w_{N(Fakestar2)^+}": 1 / 1.65,
    R"m_{N(1650)^{+}}": 1.6,
    R"w_{N(1650)^{+}}": 1 / 1.6,
}

In [ ]:
original_parameters = intensity_func_fvector.parameters
intensity_func_fvector.update_parameters(initial_parameters_fvector)
intensity_func_rel_bw.update_parameters(initial_parameters_bw)
compare_model(
    "m_01", data_real, phsp_real, intensity_func_fvector, intensity_func_rel_bw
)

In [ ]:
minuit2 = Minuit2(
    callback=CSVSummary("fit_traceback.csv"),
    use_analytic_gradient=False,
)

fit_result_BW = minuit2.optimize(estimator_bw, initial_parameters_bw)
display("Fit Breit-Wigner:", fit_result_BW)
fit_result_F = minuit2.optimize(estimator_fvector, initial_parameters_fvector)
display("Fit F vector:", fit_result_F)

In [ ]:
optimized_parameters_BW = fit_result_BW.parameter_values
optimized_parameters_F = fit_result_F.parameter_values
intensity_func_fvector.update_parameters(optimized_parameters_F)
intensity_func_rel_bw.update_parameters(optimized_parameters_BW)
compare_model(
    "m_01", data_real, phsp_real, intensity_func_fvector, intensity_func_rel_bw
)

### Parameters for $F$ vector v.s. sum of Breit-Wigners

In [ ]:
for p in optimized_parameters_F:
    print(p)
    print(f"  initial:   {initial_parameters_fvector[p]:.3}")
    print(f"  optimized F vector: {optimized_parameters_F[p]:.3}")
    print(f"  original:  {original_parameters[p]:.3}")
latest_parameters_F = CSVSummary.load_latest_parameters("fit_traceback.csv")
latest_parameters_F

In [ ]:
for p in optimized_parameters_BW:
    print(p)
    print(f"  initial:   {initial_parameters_bw[p]:.3}")
    print(f"  optimized Breit-Wigner: {optimized_parameters_BW[p]:.3}")
    print(f"  original:  {original_parameters[p]:.3}")
latest_parameters_BW = CSVSummary.load_latest_parameters("fit_traceback.csv")
latest_parameters_BW